# Generate question and answer pairs from text file

In [1]:
! pip install openai
! pip install clean-text

import dataclasses
import logging
import math
import ast
import re
import os
import io
import sys
import time
import json
import tqdm
import copy
import pandas as pd

from typing import Optional, Sequence, Union
from cleantext import clean
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


## Load and preprocess

In [2]:
dir_cur = os.getcwd()
fname = "umich.txt"
with open(os.path.join(dir_cur, fname), "r") as file:
    context = file.read()


#### Personal Identification Information (PII) removal and other preprocessing using cleantext

In [3]:

f_clean = lambda context_raw : clean(context_raw,
    fix_unicode=True,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
    no_urls=False,                  # replace all URLs with a special token
    no_emails=False,                # replace all email addresses with a special token
    no_phone_numbers=False,         # replace all phone numbers with a special token
    no_numbers=False,               # replace all numbers with a special token
    no_digits=False,                # replace all digits with a special token
    no_currency_symbols=False,      # replace all currency symbols with a special token
    no_punct=False,                 # remove punctuations
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="<URL>",
    replace_with_email="<EMAIL>",
    replace_with_phone_number="<PHONE>",
    replace_with_number="<NUMBER>",
    replace_with_digit="0",
    replace_with_currency_symbol="<CUR>",
    lang="en"                       # set to 'de' for German special handling
)
pii_remove = False
if pii_remove:
    context = f_clean(context)


#### Dataset customized clenaup and split into list of paragraphs

In [4]:
context = context.lower() # Lowercase
context = context.strip() # Remove leading/trailing whitespace
context = re.sub(r'[ \t]+', ' ', context) # Remove extra space and tabs while MAINTAINING NEW LINE CHARACTERS
context = re.compile('<.*?>').sub('', context) # Remove HTML tags/markups:

paragraphs = re.split(r'\n{2,}', context) # split it into paragraphs where there are 2+ consecutive newline characters
para_len_l = len(paragraphs)


In [5]:
paragraphs

['green card application process\nthere are essentially three steps in the employment-based green card application process:',
 '1. labor certification (perm)\nwith limited exceptions, all eb-2 and eb-3 green card applications require that the employer obtain a labor certification from the u.s. department of labor. for petitions requiring this step, the labor certification process is often the hardest and most arduous step. prior to being able to file the labor certification application, the employer must obtain a prevailing wage from the department of labor and prove that there are no minimally qualified u.s. workers available for the positions through the completion of a competitive recruitment process.',
 'in the case of positions that contain teaching duties, the employer must document that the selected applicant is the “best qualified” for the position. this process is commonly called “special handling.”',
 'in both the “basic” and the “special handling” process, the employer must 

## Question answer generation with lmqg

In [6]:
! pip install lmqg
from pprint import pprint
from lmqg import TransformersQG

# Download the en_core_web_sm model explicitly
! python -m spacy download en_core_web_sm  # spacy is a counterpart of nltk

# initialize model
model = TransformersQG(model='lmqg/t5-base-squad-qg-ae', max_length=1024) # max length of a paragraph
# paragraph to generate pairs of question and answer

context = context

question_answer = model.generate_qa(paragraphs)
# the output is a list of tuple (question, answer)
pprint(question_answer)
# [
#     ('Who was an English painter who specialised in watercolour landscapes?', 'William Turner'),
#     ('What is William Turner often known as?', 'William Turner of Oxford or just Turner of Oxford'),
#     ("What did many of Turner's paintings depict?", 'the countryside around Oxford'),
#     ("What is one of Turner's best known pictures?", 'a view of the city of Oxford from Hinksey Hill')
# ]

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/e3/43/ea958505875b22961e1277587f66b79f9e1f9d97d7998850ed089ae0d0bd/torch-2.1.2-cp310-none-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for transformers>=4.26.1 from https://files.pythonhosted.org/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 5.4 MB/s eta 0:00:00
  Using cached sentencepiece-0.1.99-cp310-cp310-macosx_11_0_arm64.whl (1.2 MB)
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Using 

/Users/joseortiz/anaconda3/envs/pykoi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 64.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Obtaining dependency information for spacy<3.8.0,>=3.7.2 from https://files.pythonhosted.org/packages/df/d2/440527cb9099be67ef0e121c71feee1f5a59c956cb10d35afdf4abc35ece/spacy-3.7.2-cp310-cp310-macosx_11_0_arm64.whl.metadata
  Using cached spacy-3.7.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (25 kB)
Using cached spacy-3.7.2-cp310-cp310-macosx_11_0_arm64.whl (6.6 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.0
    Uninstalling spacy-3.7.0:
      Successfully uninstalled spacy-3.7.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


/Users/joseortiz/anaconda3/envs/pykoi/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/Users/joseortiz/anaconda3/envs/pykoi/lib/python3.10/site-packages/transformers/models/auto/configuration_auto.py:1067: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/Users/joseortiz/anaconda3/envs/pykoi/lib/python3.10/site-packages/transformers/modeling_utils.py:2759: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/Users/joseortiz/anaconda3/envs/pykoi/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current ve

[[('How many steps are there in the employment-based green card application '
   'process?',
   'three')],
 [('What is the hardest and most arduous step for petitions requiring this '
   'step?',
   'labor certification'),
  ('What is often the hardest and most arduous step for petitions requiring '
   'this step?',
   'labor certification process'),
  ('What must an employer obtain before being able to file a labor '
   'certification application?',
   'prevailing wage')],
 [('What is special handling?',
   'the employer must document that the selected applicant is the “best '
   'qualified”'),
  ('What is the process called when an employer must document that an '
   'applicant is the best qualified for a position?',
   'special handling.')],
 [('What does the "basic" and "special handling" process require?',
   'the employer must complete a formal recruitment process to document that '
   'there are no minimally qualified u.s. workers available'),
  ('When must the formal recruitmen

In [7]:
 print (f"For {len(paragraphs)} paragraphs, {[len(qas) for qas in question_answer]} questions are generated respectively")

For 12 paragraphs, [1, 3, 2, 2, 3, 2, 2, 5, 2, 5, 3, 1] questions are generated respectively


In [8]:
QApair_flat = [qa for qas in question_answer for qa in qas]
len(QApair_flat)

31

## Write generated question answer pairs to csv file

In [9]:
QApair_df = pd.DataFrame(QApair_flat, columns=["Qustion", "Answer"])
dir_cur = os.getcwd()
QApair_df.to_csv(os.path.join(dir_cur, "output_qa_augment.csv"), index=False)